In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import os

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
catalog = 'glue'
database = 'spark_drill'
spark_conf = SparkConf()

# https://github.com/apache/spark/blob/v3.5.1/pom.xml
# https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws/3.3.4
spark_conf.setAll(
    [
        ('spark.master', 'local[*]'),
        ('spark.app.name', 'spark_app'),
        # aws & iceberg
        ('spark.jars.packages', 'org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,org.apache.iceberg:iceberg-aws-bundle:1.5.2'),
        # s3
        ('spark.hadoop.fs.s3a.access.key', aws_access_key_id),
        ('spark.hadoop.fs.s3a.secret.key', aws_secret_access_key),
        ('spark.hadoop.fs.s3a.endpoint', 's3.amazonaws.com'),
        ('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'),
        # iceberg
        ('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'),
        (f'spark.sql.catalog.{catalog}', 'org.apache.iceberg.spark.SparkCatalog'),
        (f'spark.sql.catalog.{catalog}.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog'),
        (f'spark.sql.catalog.{catalog}.warehouse', 's3://de-spark-practice/tpc-h/iceberg_table/'),
        (f'spark.sqk.catalog.{catalog}.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
    ]
)

spark = SparkSession.builder\
    .config(conf=spark_conf)\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/20 18:16:22 WARN Utils: Your hostname, DESKTOP-9INVMMS resolves to a loopback address: 127.0.1.1; using 172.29.67.34 instead (on interface eth0)
24/06/20 18:16:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/joeip/Drill/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/joeip/.ivy2/cache
The jars for the packages stored in: /home/joeip/.ivy2/jars
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f58d74cc-f7ed-4469-9106-5dc84e0977eb;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	foun

24/06/20 18:16:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
def estimtae_df_size(df) -> float:
    df.cache()
    nrows = df.count()
    size_mb = spark._jvm.org.apache.spark.util.SizeEstimator.estimate(df._jdf)/ 1024**2
    df.unpersist()

    return size_mb

Customer

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, FloatType, TimestampType, IntegerType

In [ ]:
customer_schema = StructType([
    StructField(name='cust_key', dataType=StringType(), nullable=True),
    StructField(name='name', dataType=StringType(), nullable=True),
    StructField(name='address', dataType=StringType(), nullable=True),
    StructField(name='nation_key', dataType=StringType(), nullable=True),
    StructField(name='phone', dataType=StringType(), nullable=True),
    StructField(name='acct_bal', dataType=FloatType(), nullable=True),
    StructField(name='mkt_segment', dataType=StringType(), nullable=True),
    StructField(name='comment', dataType=StringType(), nullable=True),
])
customer = spark.read\
    .options(delimiter = '|',
             header = False)\
    .schema(customer_schema)\
    .csv('s3a://de-spark-practice/tpc-h/raw/customer.tbl')

In [ ]:
customer_df_size_mb = estimtae_df_size(customer)
print('customer: ', round(customer_df_size_mb,2), 'MB')

In [ ]:
# unprocessed
customer.write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.customer_unprocessed')

customer table is small, can  coalesce into 1 single partition.

In [ ]:
# single partition 
customer\
    .repartition(1)\
    .write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.customer_single_partition')

In [ ]:
query = f'''
    SELECT 
        file_size_in_bytes/power(1024,2) as file_size_in_MB 
    FROM {catalog}.{database}.customer_single_partition.files;
'''
spark.sql(query).show()

_________
Orders

In [ ]:
order_schema = StructType([
    StructField(name='order_key', dataType=StringType(), nullable=True),
    StructField(name='cust_key', dataType=StringType(), nullable=True),
    StructField(name='order_status', dataType=StringType(), nullable=True),
    StructField(name='total_price', dataType=FloatType(), nullable=True),
    StructField(name='order_date', dataType=TimestampType(), nullable=True),
    StructField(name='order_priority', dataType=StringType(), nullable=True),
    StructField(name='clerk', dataType=StringType(), nullable=True),
    StructField(name='ship_priority', dataType=StringType(), nullable=True),
    StructField(name='comment', dataType=StringType(), nullable=True),
])

orders = spark.read\
    .options(delimiter = '|',
             header = False)\
    .schema(order_schema)\
    .csv('s3a://de-spark-practice/tpc-h/raw/orders.tbl')

In [ ]:
orders_df_size_mb = estimtae_df_size(orders)
print('orders: ', round(orders_df_size_mb,2), 'MB')

`order_key` is an continuous sequence, `arrow` will store this in the metadata instead of writing it as a physical column, to compare the storage size before/after sorting, we will hash the `order_key` with `md5`, and compare the size on disk.

In [ ]:
from pyspark.sql.functions import *

order_unprocessed = orders.withColumn('order_surrogate_key', md5(col('order_key')))\
    .drop('order_key')

order_unprocessed.write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.orders_unprocessed')

In [ ]:
# sort by low cardinality column

query = f'''
    SELECT 'order_status' as col, count(distinct order_status) as cardinality from {catalog}.{database}.orders_unprocessed
    union all
    SELECT 'order_date' as col, count(distinct order_date) as cardinality from {catalog}.{database}.orders_unprocessed
    union all
    SELECT 'order_priority' as col, count(distinct order_priority) as cardinality from {catalog}.{database}.orders_unprocessed
    union all
    SELECT 'ship_priority' as col, count(distinct ship_priority) as cardinality from {catalog}.{database}.orders_unprocessed
'''
spark.sql(query).show()

In [ ]:
spark.sql(query).explain()

Using **order_status** as an example.
- The table is not partitioned, Spark initialised 3 tasks to read the table, each task read in approximately the same number of records. ![alt text](order_status_task.jpg)
- A local aggregation on *order_status* is performed. This will give the distinct order_status locally.
- A shuffle happened to rearrange the records, records with same order_status will sit together in the same partition.
- AQE coalesces the number of partition to 1 after shuffling
- Another local aggregation on *order_status* is performed to remove duplicates
- The partial count counts the number of distinct records in each partition
- The last shuffle and aggregate make sure all the records are shuffle to the same partiton and a global aggregate is performed.

In [ ]:
# single partition, sort by columns in order of cardinality (low to high)
order_sorted = order_unprocessed.repartition(1)\
    .orderBy('order_status', 'order_priority', 'order_date')

In [ ]:
order_sorted.write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.orders_single_partition_sorted')

In [ ]:
query = f'''
    SELECT 
        'unsorted' as desc, round(sum(file_size_in_bytes/power(1024,2)),2) as file_size_in_MB 
    FROM {catalog}.{database}.orders_unprocessed.files

    union all 

    SELECT 
        'sorted' , round(sum(file_size_in_bytes/power(1024,2)),2) 
    FROM {catalog}.{database}.orders_single_partition_sorted.files;
'''
spark.sql(query).show()

The sorted version is slightly better than the unsorted version.

We will rewrite the table with the originl `order_key` to perform further analysis.

In [ ]:
orders.write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.orders_unprocessed')

In [ ]:
orders.repartition(1)\
    .orderBy('order_status', 'order_priority', 'order_date')\
    .write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.orders_single_partition_sorted')

In [ ]:
query = f'''
    SELECT 
        'unsorted' as desc, round(sum(file_size_in_bytes/power(1024,2)),2) as file_size_in_MB 
    FROM {catalog}.{database}.orders_unprocessed.files

    union all 

    SELECT 
        'sorted' , round(sum(file_size_in_bytes/power(1024,2)),2) 
    FROM {catalog}.{database}.orders_single_partition_sorted.files;
'''
spark.sql(query).show()

__________
customer & order

In [75]:
# dafault shuffle partiton count
spark.conf.set("spark.sql.shuffle.partitions", 200)

In [76]:
import time

# all time top 10 customer by spending
unprocessed_file_execution_time = []

for i in range(10):
    unprocessed_query = f'''
        with cte as (
            select
                c.name,
                sum(total_price) as total_price,
                row_number() over (partition by c.name order by sum(total_price)) as row_num
            from {catalog}.{database}.customer_unprocessed c join {catalog}.{database}.orders_unprocessed o 
            on c.cust_key = o.cust_key
            group by c.name
        )

        select
            name,
            total_price
        from cte
        where row_num <= 10
        order by total_price desc;
    '''
    start_time = time.time()

    spark.sql(unprocessed_query).collect()

    end_time = time.time()
    elapsed_time = end_time - start_time
    unprocessed_file_execution_time.append(elapsed_time)

In [77]:
# all time top 10 customer by spending
processed_file_execution_time = []
for i in range(10):
    processed_query = f'''
        with cte as (
            select
                c.name,
                sum(total_price) as total_price,
                row_number() over (partition by c.name order by sum(total_price)) as row_num
            from {catalog}.{database}.customer_single_partition c join {catalog}.{database}.orders_single_partition_sorted o 
            on c.cust_key = o.cust_key
            group by c.name
        )

        select
            name,
            total_price
        from cte
        where row_num <= 10
        order by total_price desc;
    '''
    start_time = time.time()

    spark.sql(processed_query).collect()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    processed_file_execution_time.append(elapsed_time)

In [78]:
# setting shuffle partition to 1 for processed file
spark.conf.set("spark.sql.shuffle.partitions", 2)

In [79]:
# all time top 10 customer by spending
processed_file_tuned_shuffle_partition_execution_time = []
for i in range(10):
    processed_query = f'''
        with cte as (
            select
                c.name,
                sum(total_price) as total_price,
                row_number() over (partition by c.name order by sum(total_price)) as row_num
            from {catalog}.{database}.customer_single_partition c join {catalog}.{database}.orders_single_partition_sorted o 
            on c.cust_key = o.cust_key
            group by c.name
        )

        select
            name,
            total_price
        from cte
        where row_num <= 10
        order by total_price desc;
    '''
    start_time = time.time()

    spark.sql(processed_query).collect()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    processed_file_tuned_shuffle_partition_execution_time.append(elapsed_time)

In [87]:
unprocessed_file_tuned_shuffle_partition_execution_time = []

for i in range(10):
    unprocessed_query = f'''
        with cte as (
            select
                c.name,
                sum(total_price) as total_price,
                row_number() over (partition by c.name order by sum(total_price)) as row_num
            from {catalog}.{database}.customer_unprocessed c join {catalog}.{database}.orders_unprocessed o 
            on c.cust_key = o.cust_key
            group by c.name
        )

        select
            name,
            total_price
        from cte
        where row_num <= 10
        order by total_price desc;
    '''
    start_time = time.time()

    spark.sql(unprocessed_query).collect()

    end_time = time.time()
    elapsed_time = end_time - start_time
    unprocessed_file_tuned_shuffle_partition_execution_time.append(elapsed_time)

In [98]:
import pandas as pd

unprocessed_file_execution_time_df = pd.DataFrame(unprocessed_file_execution_time)\
    .reset_index()\
    .rename(columns = {0:'unprocessed_time'})
processed_file_execution_time_df = pd.DataFrame(processed_file_execution_time)\
    .reset_index()\
    .rename(columns = {0:'processed_time'})
processed_file_tuned_shuffle_partition_execution_time_df = pd.DataFrame(processed_file_tuned_shuffle_partition_execution_time)\
    .reset_index()\
    .rename(columns = {0:'processed_tuned_shuffle_partition_time'})
unprocessed_file_tuned_shuffle_partition_execution_time_df = pd.DataFrame(unprocessed_file_tuned_shuffle_partition_execution_time)\
    .reset_index()\
    .rename(columns = {0:'tuned_shuffle_partition_time'})

combine_execution_time_df = unprocessed_file_execution_time_df\
    .merge(processed_file_execution_time_df, on = 'index')\
    .merge(processed_file_tuned_shuffle_partition_execution_time_df, on = 'index')\
    .merge(unprocessed_file_tuned_shuffle_partition_execution_time_df, on='index')\
    .rename(columns={'index':'iteration'})

In [101]:
display(combine_execution_time_df)

,iteration,unprocessed_time,processed_time,processed_tuned_shuffle_partition_time,tuned_shuffle_partition_time
0,0,6.597296,4.390409,3.133622,5.283380
1,1,4.458177,3.692109,3.119999,4.169274
2,2,5.313872,4.038579,3.044740,5.348646
3,3,4.392641,3.247443,3.273716,4.352708
4,4,4.595046,3.264681,3.071440,4.671934
5,5,4.126972,3.268821,3.215186,4.797653
6,6,4.913410,3.140642,3.474682,4.682474
7,7,4.642701,3.223811,4.720748,3.828924
8,8,5.243189,3.205679,3.058208,4.179294
9,9,5.937499,3.074231,3.113074,4.465204


In [100]:
print('unrpocessed file: ', combine_execution_time_df['unprocessed_time'].mean())
print('unprocessed file & tuned shuffle partition: ', combine_execution_time_df['tuned_shuffle_partition_time'].mean())
print('processed file: ', combine_execution_time_df['processed_time'].mean())
print('processed file & tuned shuffle partition: ', combine_execution_time_df['processed_tuned_shuffle_partition_time'].mean())

unrpocessed file:  5.022080206871033
unprocessed file & tuned shuffle partition:  4.5779492378234865
processed file:  3.4546404838562013
processed file & tuned shuffle partition:  3.3225415468215944


The query execution time for single partition table is ~33% faster than table with small files. Tuning the shuffle partition only provides a marginal benefit.

____________
line item

In [9]:
customer_schema = StructType([
    StructField(name='order_key', dataType=StringType(), nullable=True),
    StructField(name='part_key', dataType=StringType(), nullable=True),
    StructField(name='supp_key', dataType=StringType(), nullable=True),
    StructField(name='line_number', dataType=IntegerType(), nullable=True),
    StructField(name='quantity', dataType=IntegerType(), nullable=True),
    StructField(name='extended_price', dataType=FloatType(), nullable=True),
    StructField(name='discount', dataType=FloatType(), nullable=True),
    StructField(name='tax', dataType=FloatType(), nullable=True),
    StructField(name='return_flag', dataType=StringType(), nullable=True),
    StructField(name='line_status', dataType=StringType(), nullable=True),
    StructField(name='ship_date', dataType=TimestampType(), nullable=True),
    StructField(name='commit_date', dataType=TimestampType(), nullable=True),
    StructField(name='recipt_date', dataType=TimestampType(), nullable=True),
    StructField(name='ship_instruct', dataType=StringType(), nullable=True),
    StructField(name='ship_mode', dataType=StringType(), nullable=True),
    StructField(name='comment', dataType=StringType(), nullable=True),
])
line_item = spark.read\
    .options(delimiter = '|',
             header = False)\
    .schema(customer_schema)\
    .csv('s3a://de-spark-practice/tpc-h/raw/lineitem.tbl')

In [10]:
line_item.write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.line_item_unprocessed')

In [11]:
line_item.repartition(1)\
    .write\
    .format('iceberg')\
    .mode('overwrite')\
    .saveAsTable(f'{catalog}.{database}.line_item_single_partition')

__________
Most expensive order line items

In [25]:
spark.sql(f'''

    with cte as (
        select
        o.order_key,
        o.total_price,
        dense_rank() over(order by o.total_price) as price_rank
    from {catalog}.{database}.orders_single_partition_sorted o
    )
    
    select
        c.order_key,
        c.total_price,
        l.part_key
    from cte c join {catalog}.{database}.line_item_single_partition l on c.order_key = l.order_key
    where c.price_rank = 1
    
''').show()

24/06/20 22:53:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:53:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:53:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:53:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:53:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:53:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+-----------+--------+
|order_key|total_price|part_key|
+---------+-----------+--------+
|  2159139|     857.71|   32021|
+---------+-----------+--------+



In [23]:
spark.sql(f'''

    with cte as (
        select
        o.order_key,
        o.total_price,
        dense_rank() over(order by o.total_price) as price_rank
    from {catalog}.{database}.orders_unprocessed o
    )
    
    select
        c.order_key,
        c.total_price,
        l.part_key
    from cte c join {catalog}.{database}.line_item_unprocessed l on c.order_key = l.order_key
    where c.price_rank = 1
    
''').show()

24/06/20 22:51:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:51:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/20 22:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+-----------+--------+
|order_key|total_price|part_key|
+---------+-----------+--------+
|  2159139|     857.71|   32021|
+---------+-----------+--------+

